In [ ]:
# Función de coste binaria con valores {-1, 1}
# g(z) = "\
#     +11 * z_0 - 17.5 * z_1 - 28 * z_2 - 17 * z_3 + 11.5 * z_4 \
#     +13.5 * (-z_0*z_2 + z_1*z_2 + z_2*z_3 - z_2*z_4 + z_0*z_1 - z_0*z_3 - z_1*z_4) \
#     +(6.75*z_3**2 + 6.75*z_4**2 + 13.5*(z_0**2 + z_1**2 + z_2**28)) \
#     +26.5"

# Grafo
edges = {(0, 1): 5, (0, 2): 8, (1, 2): 2, (1, 3): 7, (2, 3): 4}
num_nodes = 4

# Hp = 11*Z0 - 17.5*Z1 - 28*Z2 - 17*Z3 + 11.5*Z4 +
#      13.5*(-Z0*Z2 + Z1*Z2 + Z2*Z3 - Z2*Z4 + Z0*Z1 - Z0*Z3 - Z1*Z4)

# Coeficientes lineales de Hp (11 * Z0 - 17.5 * Z1...)
linear_coefs = [11, -17.5, -28, -17, 11.5]

# Coeficientes cuadráticos Ej: {(2, 3): 13.5} -> 13.5 * Z2 * Z3
quadra_coefs = {(0, 2): -13.5, (1, 2): 13.5, (2, 3): 13.5, (2, 4): -13.5,
                (0, 1): 13.5, (0, 3): -13.5, (1, 4): -13.5}

In [ ]:
from qiskit import QuantumCircuit

def generate_qaoa_circuit(theta):
    assert len(theta) % 2 == 0, "Error in parameters (Beta, Gamma)"

    nqubits = len(edges)  # Tantos qubits como aristas tenga el grafo
    circuit = QuantumCircuit(nqubits)

    layers = int(len(theta) / 2)
    beta = theta[:layers]
    gamma = theta[layers:]

    # |v0>
    for i in range(nqubits):
        circuit.h(i)

    for p in range(layers):
        circuit.barrier()

        # Hp
        for q_idx, coef in enumerate(linear_coefs):
            circuit.rz(coef, q_idx)
        for q_idxs, coef in quadra_coefs.items():
            circuit.rzz(coef * gamma[p], q_idxs[0], q_idxs[1])

        circuit.barrier()

        # Hm
        for q_idx in range(nqubits):
            circuit.rx(beta[p] * 2, q_idx)

    return circuit

In [ ]:
# Statevector
from qiskit import Aer, assemble
import numpy as np
backend = Aer.get_backend('aer_simulator')
shots = 1

theta = [0.28517317, -5.05969577]  # Resultado obtenido en el paper
qc = generate_qaoa_circuit(theta)

qc.save_statevector()

qobj = assemble(qc)
vector = backend.run(qobj, shots=shots).result().get_statevector()
vector = np.asarray(vector)
vector

In [ ]:
import itertools
from math import sqrt
from qiskit.visualization import plot_histogram

paths = [''.join(bit_arr) for bit_arr in
         itertools.product(['0', '1'], repeat=int(sqrt(vector.size)))]

counts = {}
for amplitude, path in zip(vector, paths):
    probability = amplitude.real ** 2 + amplitude.imag ** 2
    counts[path] = probability

print(sum(counts.values()))
plot_histogram(counts, figsize=(12, 7))